In [2]:
import pandas as pd

In [10]:
gpus_dataset = pd.read_csv("gpus.csv")

In [11]:
gpus_dataset.head()

,name,type,tdp_watts,TFLOPS32,TFLOPS16,GFLOPS32/W,GFLOPS16/W,memory,source
0,A100 PCIe 40/80GB,gpu,250,NaN,312.00,NaN,NaN,40.0,https://www.nvidia.com/content/dam/en-zz/Solut...
1,A100 SXM4 80 GB,gpu,400,NaN,312.00,NaN,NaN,80.0,https://www.nvidia.com/content/dam/en-zz/Solut...
2,AGX Xavier,gpu,30,16.00,32.00,533.33,1066.67,NaN,https://elinux.org/Jetson_AGX_Xavier
3,AMD RX480,gpu,150,5.80,5.80,38.67,38.67,8.0,techpowerup.com
4,GTX 1080 Ti,gpu,250,11.34,0.17,45.36,0.68,11.0,techpowerup.com


In [12]:
gpus_dataset = gpus_dataset[["name","type","tdp_watts"]]
gpus_dataset.head()

,name,type,tdp_watts
0,A100 PCIe 40/80GB,gpu,250
1,A100 SXM4 80 GB,gpu,400
2,AGX Xavier,gpu,30
3,AMD RX480,gpu,150
4,GTX 1080 Ti,gpu,250
